In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import cv2
import csv
import json
import os.path
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, Lambda, MaxPooling2D, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
import tensorflow as tf
%matplotlib inline

In [3]:
# Import data
def import_data(csv_file):
    data = pd.read_csv(csv_file, header=None)
    data.columns = ['center','left','right','steering','throttle','brake','speed']
    data.drop(['left','right','throttle','brake'], axis = 1)
    return data

df = import_data('driving_log.csv')


In [9]:
# Extract data

width = 200
height = 66
depth = 3

def extract_data(data):
    image_path = np.copy(data['center'])
    features = np.zeros(shape=[len(image_path), height, width, depth])
    labels = np.copy(df['steering']).astype(np.float32)
    for i, path in enumerate(image_path):
        image = mpimg.imread(path)
        shape = image.shape
        image = image[int(image.shape[0]/3):image.shape[0], :]
        image = cv2.resize(image, (width, height))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        features[i, :, :, :] = image
    return features, labels

X_train, y_train = extract_data(df)


In [17]:
patch_size = 5
depth1 = 32
depth2 = 64
num_hidden = 1024
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape = X_train.shape[1:] ))
    model.add(Convolution2D(depth1, patch_size, patch_size,
                           border_mode = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(depth2, patch_size, patch_size,
                           border_mode = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(num_hidden, W_regularizer=l2(1e-6)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, W_regularizer=l2(1e-6)))
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model = define_model()

history = model.fit(X_train, y_train,
                    batch_size = 128, 
                    nb_epoch = 5,
                    validation_split = 0.2)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_5 (Lambda)                (None, 66, 200, 3)    0           lambda_input_4[0][0]             
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 66, 200, 32)   2432        lambda_5[0][0]                   
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 66, 200, 32)   0           convolution2d_6[0][0]            
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 33, 100, 32)   0           activation_8[0][0]               
___________________________________________________________________________________________

KeyboardInterrupt: 